### Running the code in a clean notebook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import pandas_datareader as pdr

from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from pykalman import KalmanFilter

data = pd.read_csv("/Users/Sella/Desktop/2024Spring/FinancialAnalytics/FinalProject/intermediate/backtesting_data.csv")

In [5]:
import sys
sys.path.append('/Users/Sella/Desktop/2024Spring/FinancialAnalytics/FinalProject/intermediate')

import importlib
import functions_logic_changed
importlib.reload(functions_logic_changed)
#from functions import *
from functions_logic_changed import *

In [6]:
UPPERBOUND = 1
LOWERBOUND = -1
LONG = -50000000  # Using the minus as we go long when the spread is negative
MIN_CONSECUTIVE_DAYS = 3  # t, the minimum number of consecutive days to enter a trade

In [7]:
data

,date,FEZ Price,EWJ Price,Z-Score,long_size,long_etf
0,2002-10-22,12.568131,20.394314,-1.084152,5.420759e+07,EWJ
1,2002-10-23,12.250083,20.727160,-0.957287,4.786433e+07,NaN
2,2002-10-24,12.198788,20.394314,-1.012163,5.060816e+07,EWJ
3,2002-10-25,12.337286,20.848198,-0.950692,4.753459e+07,NaN
4,2002-10-28,12.527097,20.848198,-0.987688,4.938438e+07,NaN
...,...,...,...,...,...,...
5369,2024-02-22,50.344189,68.900002,1.007151,-5.035756e+07,FEZ
5370,2024-02-23,50.374134,69.010002,1.022755,-5.113773e+07,FEZ
5371,2024-02-26,50.434017,69.000000,1.009133,-5.045667e+07,FEZ
5372,2024-02-27,50.673565,69.180000,0.997527,-4.987635e+07,NaN


In [8]:
## Adding columns to the dataframe
data = data.copy()
data['long_size'] = data['Z-Score'] * LONG

# Long versus short
data['long_etf'] = np.where(
    (data['Z-Score'] > UPPERBOUND) & (data['Z-Score'] > 0), 'FEZ',
    np.where(
        (data['Z-Score'] < LOWERBOUND) & (data['Z-Score'] < 0), 'EWJ',
        np.nan
    )
)

In [9]:
processed_data = identify_trade_periods(data, UPPERBOUND, LOWERBOUND, MIN_CONSECUTIVE_DAYS)
processed_data

,date,FEZ Price,EWJ Price,Z-Score,long_size,long_etf,trade_condition,condition_cumsum,condition_count,execute_trade,entry_zscore,in_trade
0,2002-10-22,12.568131,20.394314,-1.084152,5.420759e+07,EWJ,True,1,1,False,NaN,False
1,2002-10-23,12.250083,20.727160,-0.957287,4.786433e+07,nan,False,2,0,False,NaN,False
2,2002-10-24,12.198788,20.394314,-1.012163,5.060816e+07,EWJ,True,3,1,False,NaN,False
3,2002-10-25,12.337286,20.848198,-0.950692,4.753459e+07,nan,False,4,0,False,NaN,False
4,2002-10-28,12.527097,20.848198,-0.987688,4.938438e+07,nan,False,4,0,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
5369,2024-02-22,50.344189,68.900002,1.007151,-5.035756e+07,FEZ,True,181,1,False,NaN,False
5370,2024-02-23,50.374134,69.010002,1.022755,-5.113773e+07,FEZ,True,181,2,False,NaN,False
5371,2024-02-26,50.434017,69.000000,1.009133,-5.045667e+07,FEZ,True,181,3,True,1.009133,True
5372,2024-02-27,50.673565,69.180000,0.997527,-4.987635e+07,nan,False,182,0,False,1.009133,False


In [10]:
processed_data = process_trading_periods(data)
print(processed_data[['Z-Score', 'execute_trade', 'hedge_ratio']].head(10))

    Z-Score  execute_trade  hedge_ratio
0 -1.084152          False          NaN
1 -0.957287          False          NaN
2 -1.012163          False          NaN
3 -0.950692          False          NaN
4 -0.987688          False          NaN
5 -0.937491          False          NaN
6 -0.924812          False          NaN
7 -0.998889          False          NaN
8 -0.957214          False          NaN
9 -0.980515          False          NaN


In [11]:
processed_data = calculate_trade_pnl(processed_data, LONG)

In [12]:
processed_data

,date,FEZ Price,EWJ Price,Z-Score,long_size,long_etf,trade_condition,condition_cumsum,condition_count,execute_trade,...,EWJ_return,PnL_per_trade,daily_PnL_per_trade,long_shares,short_shares,long_allocation,short_allocation,capital_layover,return,prev_hedge_ratio
0,2002-10-22,12.568131,20.394314,-1.084152,5.420759e+07,EWJ,True,1,1,False,...,NaN,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.579447
1,2002-10-23,12.250083,20.727160,-0.957287,4.786433e+07,nan,False,2,0,False,...,0.016321,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.579447
2,2002-10-24,12.198788,20.394314,-1.012163,5.060816e+07,EWJ,True,3,1,False,...,-0.016058,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.579447
3,2002-10-25,12.337286,20.848198,-0.950692,4.753459e+07,nan,False,4,0,False,...,0.022255,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.579447
4,2002-10-28,12.527097,20.848198,-0.987688,4.938438e+07,nan,False,4,0,False,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.579447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5369,2024-02-22,50.344189,68.900002,1.007151,-5.035756e+07,FEZ,True,181,1,False,...,0.016374,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.717333
5370,2024-02-23,50.374134,69.010002,1.022755,-5.113773e+07,FEZ,True,181,2,False,...,0.001597,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.718393
5371,2024-02-26,50.434017,69.000000,1.009133,-5.045667e+07,FEZ,True,181,3,True,...,-0.000145,0.000000,0.000000,-1.000449e+06,525573.684315,-5.045667e+07,3.626458e+07,-1.419208e+07,0.000000,0.718727
5372,2024-02-27,50.673565,69.180000,0.997527,-4.987635e+07,nan,False,182,0,False,...,0.002609,-334258.738733,-334258.738733,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.023552,0.719177


In [13]:
processed_data.columns

Index(['date', 'FEZ Price', 'EWJ Price', 'Z-Score', 'long_size', 'long_etf',
       'trade_condition', 'condition_cumsum', 'condition_count',
       'execute_trade', 'entry_zscore', 'in_trade', 'hedge_ratio',
       'FEZ_return', 'EWJ_return', 'PnL_per_trade', 'daily_PnL_per_trade',
       'long_shares', 'short_shares', 'long_allocation', 'short_allocation',
       'capital_layover', 'return', 'prev_hedge_ratio'],
      dtype='object')

## Risk Part

In [14]:
result_df = create_trade_summary_df(processed_data)
result_df

,date,long_etf,hedge_ratio,long_allocation,short_allocation,capital_layover,entry_zscore,PnL_per_trade,daily_PnL_per_trade,return
0,2002-11-26,EWJ,0.579447,5.155867e+07,-2.987552e+07,2.168315e+07,-1.031173,1.534662e+06,1.705180e+05,0.070777
1,2002-12-03,EWJ,0.602062,5.329034e+07,-3.180987e+07,2.148047e+07,-1.065807,-1.247268e+06,-1.247268e+06,-0.058065
2,2002-12-09,EWJ,0.600288,5.538349e+07,-3.335115e+07,2.203235e+07,-1.107670,-2.022921e+06,-2.022921e+06,-0.091816
3,2002-12-13,EWJ,0.591130,5.391342e+07,-3.204049e+07,2.187293e+07,-1.078268,-6.979176e+05,-6.979176e+05,-0.031908
4,2002-12-19,EWJ,0.607825,6.024085e+07,-3.614768e+07,2.409317e+07,-1.204817,2.446156e+05,2.446156e+05,0.010153
...,...,...,...,...,...,...,...,...,...,...
153,2023-09-28,FEZ,0.678751,-5.042076e+07,3.442084e+07,-1.599992e+07,1.008415,-7.249049e+05,-3.624525e+05,0.045307
154,2024-01-17,FEZ,0.687066,-6.382321e+07,4.560317e+07,-1.822004e+07,1.276464,9.686816e+05,9.686816e+05,-0.053166
155,2024-01-23,FEZ,0.686782,-6.626861e+07,4.588578e+07,-2.038283e+07,1.325372,5.340514e+05,5.340514e+05,-0.026201
156,2024-02-21,FEZ,0.714107,-5.091784e+07,3.633664e+07,-1.458120e+07,1.018357,-6.299826e+05,-2.099942e+05,0.043205


In [15]:
risk_metrics = calculate_risk_metrics(result_df)
risk_metrics

{'Total Return': 47685697.01233424,
 'Average Return per Trade': 301808.2089388243,
 'Average Return Rate': -0.09365261167155019,
 'Sharpe Ratio': 1.667247635958641,
 'Maximum Drawdown': -44870925.43881531,
 'Win Rate': 0.5063291139240507,
 'Average Win': 2414673.080617058,
 'Average Loss': -1865232.6850901337,
 'VaR (95%)': -1837328.671105332,
 'CVaR (95%)': -2959601.664431384}